# Renamed ProcDump
Detects the execution of a renamed ProcDump executable often used by attackers or malware

## Rule Content
```
- title: Renamed ProcDump
  id: 4a0b2c7e-7cb2-495d-8b63-5f268e7bfd67
  status: experimental
  description: Detects the execution of a renamed ProcDump executable often used by
    attackers or malware
  references:
  - https://docs.microsoft.com/en-us/sysinternals/downloads/procdump
  author: Florian Roth
  date: 2019/11/18
  tags:
  - attack.defense_evasion
  - attack.t1036
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      OriginalFileName: procdump
    filter:
      Image:
      - '*\procdump.exe'
      - '*\procdump64.exe'
    condition: selection and not filter
  falsepositives:
  - Procdump illegaly bundled with legitimate software
  - Weird admins who renamed binaries
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(file_name_original:"procdump" AND (NOT (process_path.keyword:(*\\procdump.exe OR *\\procdump64.exe))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()